In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer, GenerationConfig

model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
import platform
import os
import pandas as pd


TEST_NAME = "llm1"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + "output/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_new2_temp.csv"
VALID_PATH = DATA_PATH + "dev_new2.csv"
TEST_PATH = DATA_PATH + "test_kkh.csv"
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

train_df = pd.read_csv(os.path.join(TRAIN_PATH))
val_df = pd.read_csv(os.path.join(VALID_PATH))
test_df = pd.read_csv(os.path.join(TEST_PATH))

sampled_df = train_df.sample(n=5, random_state=42)  # random_state는 결과를 재현 가능하게 합니다.
combined_string = '\n'.join(sampled_df.apply(lambda row: f"{row['fname']}, {row['dialogue']}, {row['summary']}, {row['topic']}", axis=1).tolist())
print(combined_string)

train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드, 우리가 여기 앉아

In [4]:

for index, row in test_df.iterrows():
    row['summary'] = '"' + gen(f"{combined_string} \n\n\n\
        - 위와 같은 대화문과 요약문이 존재하니, 참고해줘.\n\
        - 아래 대화문에서 핵심 키워드를 3개 정도 추출해서 출력해주고, 핵심 키워드를 꼭 포함해서 요약문을 작성해줘.\n\
        - 요약문은 한 문장 또는 두 문장으로 표현해주고, 더불어, 요약문의 len() 길이는 101을 넘지 않도록 요약해줘.\n\
        - 그리고, 내가 적었던 문장을 다시 출력하지 말아줘.\n\
    \n{row['dialogue']}") + '"'
    print(f"@ row['fname']: {row['fname']}")
    print(f"@ row['dialogue']: {row['dialogue']}")
    print(f"@ row['summary']: {row['summary']}")
    print('=====================================')

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (3915 > 1024). Running this sequence through the model will result in indexing errors


대화문: test_0
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 전에 모든 직원에게 타이핑하여 배포해 주세요.
<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.


/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 직원, 메시지, 제한, 이메일, 공식 메모, 근무 시간, 경고, 직무 정지, 해고, 타이핑, 배포

요약문: 모든 직원들에게 이메일 통신과 공식 메모로 통신을 제한하고, 메시지 프로그램 사용은 금지된다. 이 사무실 내에서 직원들은 이 새로운 정책에 따라 즉시 메시지를 계속 사용하지 않아야 하며, 두 번째 위반 시에는 해고될 것이다. 모든 직원에게 오후 4시 전에 타이핑하여 배포해야 한다.</s>
"<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


...

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 올림픽 스타디움, 완성, 좌석, 등반 금지

요약문: 이 올림픽 스타디움은 6월에 완공될 예정이며, 총 5000개의 좌석이 있습니다. 이 공원에는 트랙이와 점프 피트가 있습니다. 하지만 등반은 금지되어 있습니다.</s>
"<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 창업, 사업 계획서, 회사, 투자자, 재무 분석, 회사 설명, 상품 소개, 시장 분석, 예측

요약문: 이런 회사에서 일하는 건 나에게 아무런 도움이 안 돼. 그래서 내 회사를 창업하기로 결정했어. 사업 계획서를 작성하고, 투자자를 모집하고, 나 자신을 위해 일하기로 했어. 사업 계획서에는 사업 아이디어를 두 페이지 이내로 강조하는 요약문이 포함되어야 하고, 회사에 대해 설명해야 해. 법적 구조, 역사 등의 정보를 포함해서 말이야. 그리고 상품이나 서비스를 소개하고 설명해야 해. 그것들이 무엇이고 

KeyboardInterrupt: 

In [5]:
summary_list = []
for index, row in test_df.iterrows():
    row['summary'] = '"' + gen(f"{combined_string} \n\n\n\
        - 위와 같은 대화문과 요약문이 존재하니, 참고해줘.\n\
        - 아래 대화문에서 핵심 키워드를 3개 정도 추출해서 출력해주고, 핵심 키워드를 꼭 포함해서 요약문을 작성해줘.\n\
        - 요약문은 한 문장 또는 두 문장으로 표현해주고, 더불어, 요약문의 len() 길이는 101을 넘지 않도록 요약해줘.\n\
        - 요약문의 형태는 대화문을 제 3자가 설명하듯이 적어줘.\n\
        - 사람 이름 대신에, #Person1# 이나 #Person2# 와 같은 키워드 형태로 넣어줘.\n\
        - 그리고, 내가 적었던 문장을 다시 출력하지 말아줘.\n\
    \n{row['dialogue']}") + '"'
    print(f"@ row['fname']: {row['fname']}")
    print(f"@ row['dialogue']: {row['dialogue']}")
    print(f"@ row['summary']: {row['summary']}")
    print('=====================================')
    summary_list.append(row['summary'])

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (4006 > 1024). Running this sequence through the model will result in indexing errors


<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드,

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 대화문, 요약문, 내부 메모, 직원들, 외부 통신, 메시지 프로그램, 직무 정지, 해고

요약문: 실장님은 모든 직원들에게 내부 메모로 전달되어야 하는 내용을 전달하였습니다. 이 메모는 모든 직원에게 적용되며, 내부와 외부 통신에 대한 제한이 있습니다. 직원들은 메시지 프로그램을 사용하지 못하고, 두 번의 위반 시 직무 정지에 처해지며, 해고될 수도 있습니다. 직원들은 부서장에게 직접 문의하여 질문이 있다면 해결해 주기를 바랍니다.</s>
@ row['fname']: test_0
@ row['dialogue']: #Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


...

[/INST] 핵심 키워드: 안녕, 린팡, 영어, 낸시, 수학, 체육, 중국어, 과학
#Person1#: 안녕, 린팡!
#Person2#: 오! 안녕, 루시!
#Person1#: 다음 수업은 뭐야, 린팡?
#Person2#: 영어야.
#Person1#: 오, 좋다! 그게 너가 항상 쉽게 느끼니까 그런 거야. 나는 매우 어려워서 별로 안 좋아해.
#Person2#: 그건 너가 항상 쉽게 느끼니까 그런 거야. 나는 매우 어려워서 별로 안 좋아해.
#Person1#: 낸시도 영어가 꽤 어렵다고 해. 하지만 그녀가 제일 좋아하는 과목이래.
#Person2#: 그럼 영어가 네 가장 좋아하는 과목이라면, 두 번째로 좋아하는 과목은 뭐야?
#Person1#: 체육이야. 널 요?
#Person2#: 잘 모르겠어. 중국어와 과학 둘 다 내가 좋아하는 과목이야. 과학보다는 중국어를 좀 더 좋아하는 것 같아.
#Person1#: 널 낸시와 많이 다르네. 그녀는 과학을 전혀 좋아하지 않아.</s>
@ row['fname']: test_46
@ row['dialogue']: #Person1#: 안녕, 린팡!
#Person2#: 오! 안녕, 루시!
#Person1#: 다음 수업은 뭐야, 린팡?
#Person2#: 영어야.
#Person1#: 오, 좋다! 그게 내 가장 좋아하는 과목이야.
#Person2#: 그건 너가 항상 쉽게 느끼니까 그런 거야. 나는 매우 어려워서 별로 안 좋아해.
#Person1#: 낸시도 영어가 꽤 어렵다고 해. 하지만 그녀가 제일 좋아하는 과목이래.
#Person2#: 응, 알아. 그리고 그녀의 두 번째로 좋아하는 과목은 수학이야.
#Person1#: 수학은 내 최악이야. 안 좋아해. 항상 답을 틀려.
#Person2#: 그럼 영어가 네 가장 좋아하는 과목이라면, 두 번째로 좋아하는 과목은 뭐야?
#Person1#: 체육이야. 넌요?
#Person2#: 잘 모르겠어. 중국어와 과학 둘 다 내가 좋아하는 과목이야. 과학보다는 중국어를 좀 더 좋아하는 것 같아.


/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 바이러스, 이메일, 주소록, 자동, 프로그램, 문제, 컴퓨터

요약문:
저는 컴퓨터에 끔찍한 바이러스가 걸렸다고 말해줘서, 이제 그만하고 진지하게 대답해줘야겠습니다. IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요.</s>
@ row['fname']: test_47
@ row['dialogue']: #Person1#: 네, 토마스 부인. 마당에 낙엽을 다 치웠어요.
#Person2#: 고마워요, 제임스.
#Person1#: 앞 계단도 쓸고 쓰레기도 버렸어요. 제가 또 도와드릴 일이 있나요?
#Person2#: 제 개 오스카도 산책시켜 주실 수 있나요?
#Person1#: 물론이죠.
#Person2#: 여기 왕자님이 계시네요. 다리가 부러진 이후로 집안일을 돌보는 것이 너무 힘들었어요. 월요일과 화요일에는 딸 레아가, 목요일에는 아들 로비가 도와주는데 토요일에 오실 수 있어서 정말 기뻐요.
#Person1#: 주말에도 일해 주셔서 감사합니다. 농구 연습하러 갈 수 있도록 자전거를 사려고 저축 중이에요.
#Person2#: 자전거를 구입한 후에도 계속 일할 수 있나요?
#Person1#: 물론이죠, 하지만 다음 주말에 할아버지 댁에 가 있는 걸 잊지 마세요. 여동생 앨리슨이 대신 올 거예요.
#Person2#: 알았어요.
@ row['summary']: "<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 200

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 봄, 난방, 에어컨, 창문

요약문: 봄이 온 것 같아요. 하지만 밤에는 여전히 매우 